<a href="https://colab.research.google.com/github/ary3120-droid/myproject/blob/main/a_btest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
#데이터 불러오기
import pandas as pd

df = pd.read_csv('/content/cookie_cats.csv')
df.head()


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [41]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [42]:
#결측치 확인
df.isnull().sum()


,0
userid,0
version,0
sum_gamerounds,0
retention_1,0
retention_7,0


In [43]:
# 숫자형(int, float) 컬럼만 선택해서 컬럼명 목록으로 저장
num_cols = df.select_dtypes(include='number').columns
num_cols


Index(['userid', 'sum_gamerounds'], dtype='object')

In [44]:
#이상치 확인
df[num_cols].describe()


,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


In [45]:
# sum_gamerounds의 상위 1%(99%) 값 계산
cap = df['sum_gamerounds'].quantile(0.99)

# 상위 1%를 cap 값으로 제한하여 이상치 영향 완화
df['sum_gamerounds_capped'] = df['sum_gamerounds'].clip(upper=cap)

# cap 처리된 데이터 비율(상위 1% 이상치 비중) 확인
(df['sum_gamerounds'] >= cap).mean()


np.float64(0.010045570967634634)

In [46]:
# 원본 sum_gamerounds의 최대값 확인
df['sum_gamerounds'].max()

# cap 적용 후 sum_gamerounds_capped의 최대값 확인
df['sum_gamerounds_capped'].max()


493

In [47]:
# 원본 sum_gamerounds의 기초 통계량 확인
df['sum_gamerounds'].describe()

# cap 적용 후 sum_gamerounds_capped의 기초 통계량 확인
df['sum_gamerounds_capped'].describe()


,sum_gamerounds_capped
count,90189.000000
mean,48.993647
std,84.205426
min,0.000000
25%,5.000000
50%,16.000000
75%,51.000000
max,493.000000


중앙값/사분위 유지

25%: 5

50%: 16

75%: 51

👉 cap 전과 동일

평균 변화

원본 평균 ≈ 51.9

cap 후 평균 ≈ 49.0

👉 극단값 영향만 제거된 건강한 하락

In [49]:
# userid 기준 중복 데이터 존재 여부 확인
# A/B 테스트는 유저 단위 분석이므로, 동일 유저의 중복 관측치가 있는지 점검
df['userid'].duplicated().sum()

# 결과가 0이므로 userid 중복 없음
# → 모든 행이 서로 다른 유저를 의미하며, 추가적인 중복 제거 불필요
# → 관측치 간 독립성 가정 충족


np.int64(0)

In [50]:
# Control / Treatment 그룹별 표본 수 확인
# 두 그룹 간 샘플 수 불균형 여부 점검
df['version'].value_counts()


,count
version,
gate_40,45489
gate_30,44700


두 그룹의 표본 수가 거의 동일

표본 불균형으로 인한 편향 우려 낮음
→ A/B 테스트 설계 적절

In [51]:
# 그룹별 리텐션 평균 확인 (탐색적 분석)
# 가설 검정 전, Treatment가 Control 대비 성과 차이가 있는지 직관적으로 파악
df.groupby('version')[['retention_1', 'retention_7']].mean()


,retention_1,retention_7
version,,
gate_30,0.448188,0.190201
gate_40,0.442283,0.182000


관찰 결과 (탐색적 비교)

1일 리텐션

gate_30 > gate_40 (약 +0.6%p)

7일 리텐션

gate_30 > gate_40 (약 +0.8%p)

👉 즉,

gate을 30레벨에 배치한 그룹이 단기·중기 리텐션 모두 더 높음